In [1]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install --upgrade pyopenssl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 968.1 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 15.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: cryptography
    Found existing installation: cryptography 37.0.2
    Uninstalling cryptography-37.0.2:
      Successfully uninstalled cryptography-37.0.2
  Attempting uninstall: pyopenssl
    Found existing installation: pyOpenSSL 19.0.0
    Uninstalling pyOpenSSL-19.0.0:
      Successfully uninstalled pyOpenSSL-19.0.0
Note: you may need to restart the kernel to use updated packages.


In [9]:
import requests

In [10]:
from bs4 import BeautifulSoup

In [148]:
url = 'https://friends.fandom.com/wiki/Chandler_Bing'

In [149]:
response = requests.get(url)

In [150]:
if response.status_code == 200:
# Parse the HTML content of the page
    soup = BeautifulSoup(response.text, 'lxml')  # Use 'lxml' or 'html5lib' as per your preference

    #Extracting paragraph
    paragraphs = soup.find_all('p')
    for paragraph in paragraphs:
        print(paragraph.text)
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")    


Character
Chandler Bing






Name
Chandler Muriel Bing


Nicknames
Ringo (by Monica in "The One With The Fake Party")Mr. Big (by Monica)Sir-Limps-a-Lot (in college, created by Ross)Roland Chang (fake ID in "The One With All The Thanksgivings")Shaw-La-La (by Doug)Bing-a-Ling (by Janice)Miss Chanandler BongBing (by Doug)Toby (by Bob)'The Funny One'Skidmark (by Julie Graff in "The One With The Nap Partners")The DropperThe King of Bad ThanksgivingsChan-Chan Man (by Joey)DuckChanny (by Joey)Channy Fanny (by Frank Buffay Jr. in "The One Hundredth")Mondler (by Ross in "The One Where Phoebe Runs")Chauncey (by Jack Geller and Ross in "The One With Phoebe's Cookies")ClintGeneMark JohnsonJohn MarksonSweetie, Honey (by Monica)Mr. Sensitivity (by Monica in "The One With Phoebe's Birthday Dinner")The Boy Who Hates Thanksgiving (by Judy Geller)ChucklesBob (by Jade)Dennis (by woman at the 2 parties)Smokey Joe (by Joey on the ski trip)Little Baby Girl Chandler (By Coach Rubin)Dude (by Joey Tribbiani)

In [151]:
import nltk

# Download the nltk punkt data for sentence tokenization
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/kirti/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [152]:
sentences = []
for paragraph in paragraphs:
 sentences.extend(nltk.sent_tokenize(paragraph.text))

In [153]:
import re

In [154]:
cleaned_sentences = [re.sub(r'\[.*?\]', '', sentence) for sentence in sentences]

In [155]:
import pandas as pd
import numpy as np

In [156]:
#Create a DataFrame
data = {'Sentences': cleaned_sentences}
df = pd.DataFrame(data)

# Save DataFrame to a CSV file
df.to_csv('sentences.csv', index=False)

In [157]:
df.head()

,Sentences
0,\nCharacter\nChandler Bing\n\n\n\n\n\n\nName\n...
1,"Sensitivity (by Monica in ""The One With Phoebe..."
2,)Unnamed high school (formerly/graduated)\n\n\...
3,Unnamed high school (formerly/graduated)
4,—Chandler after being asked for advice.


In [158]:
df.isnull().sum()

Sentences    0
dtype: int64

In [159]:
cleaned_df = df.dropna(subset=df.columns, how='all')

In [160]:
cleaned_df.isnull().sum()

Sentences    0
dtype: int64

In [161]:
cleaned_df.head()

,Sentences
0,\nCharacter\nChandler Bing\n\n\n\n\n\n\nName\n...
1,"Sensitivity (by Monica in ""The One With Phoebe..."
2,)Unnamed high school (formerly/graduated)\n\n\...
3,Unnamed high school (formerly/graduated)
4,—Chandler after being asked for advice.


In [162]:
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

# Assuming 'snape_life' is your DataFrame
cleaned_df['Sentences'] = cleaned_df['Sentences'].fillna('')  # Replace NaN with an empty string

# Initialize the Sentiment Intensity Analyzer
sia = SentimentIntensityAnalyzer()

# Create a new column 'Sentiment' with sentiment scores
cleaned_df['Sentiment'] = cleaned_df['Sentences'].apply(lambda x: sia.polarity_scores(x)['compound'])

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/kirti/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [163]:
threshold_angry = -0.7
threshold_anguish = -0.5
threshold_contempt = -0.05
threshold_worry = 0
threshold_ambiguity = 0.1
threshold_affection = 0.2  
threshold_sacrifice = 0.3
threshold_protectiveness = 0.5
threshold_sarcasm = 0.6

# Create conditions and corresponding labels
conditions = [
    cleaned_df['Sentiment'] <= threshold_angry,
    (cleaned_df['Sentiment'] > threshold_angry) & (cleaned_df['Sentiment'] <= threshold_anguish),
    (cleaned_df['Sentiment'] > threshold_anguish) & (cleaned_df['Sentiment'] <= threshold_contempt),
    (cleaned_df['Sentiment'] > threshold_contempt) & (cleaned_df['Sentiment'] <= threshold_worry),
    (cleaned_df['Sentiment'] > threshold_worry) & (cleaned_df['Sentiment'] <= threshold_ambiguity),
    (cleaned_df['Sentiment'] > threshold_ambiguity) & (cleaned_df['Sentiment'] <= threshold_affection),
    (cleaned_df['Sentiment'] > threshold_affection) & (cleaned_df['Sentiment'] <= threshold_sacrifice),
    (cleaned_df['Sentiment'] > threshold_sacrifice) & (cleaned_df['Sentiment'] <= threshold_protectiveness),
    (cleaned_df['Sentiment'] > threshold_protectiveness) & (cleaned_df['Sentiment'] <= threshold_sarcasm),
    cleaned_df['Sentiment'] > threshold_sarcasm
]

labels = ['angry', 'anguish', 'contempt', 'worry','ambiguity', 'affection', 'sacrifice', 'protectiveness', 'sarcasm', 'unknown']

# Create a new column with the sentiment categories
cleaned_df['Emotion_Category'] = np.select(conditions, labels, default='unknown')

In [164]:
emotion_percentages = cleaned_df['Emotion_Category'].value_counts(normalize=True) * 100
emotion_percentages

worry             27.637131
unknown           19.198312
contempt          14.767932
protectiveness    12.658228
sacrifice          6.329114
anguish            5.907173
sarcasm            5.485232
angry              3.375527
ambiguity          2.320675
affection          2.320675
Name: Emotion_Category, dtype: float64

In [165]:
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /Users/kirti/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/kirti/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [166]:
all_text = ' '.join(cleaned_df['Sentences'].astype(str))

# Tokenize the text into words
words = word_tokenize(all_text)

# Remove stop words and punctuation
stop_words = set(stopwords.words('english'))
filtered_words = [word.lower() for word in words if word.isalnum() and word.lower() not in stop_words]

# Lemmatize the words
lemmatizer = WordNetLemmatizer()
lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

# Calculate the frequency distribution of words
freq_dist = FreqDist(lemmatized_words)

# Find the top 5 most common words
top_words = freq_dist.most_common(5)

# Print the top 5 most common words and their frequencies
for word, frequency in top_words:
    print(f"The word '{word}' appears {frequency} times.")

The word 'chandler' appears 258 times.
The word 'monica' appears 145 times.
The word 'joey' appears 109 times.
The word 'ross' appears 96 times.
The word 'one' appears 85 times.
